# Datagenerering til DigDirCamp 2020

av Bjørn Christian Weinbach, DigDir Campar.

## Bakgrunn

I vår applikasjon skal me demonstrere korleis tilgang til eins eigne persondata kan gjere det enklare å navigere i den norske offentlege sektor, spesielt for ungdom. Denne dataen er ikkje tilgjengeleg vha API i dag og vi er nøydd å generere denne dataen sjølv. Denne jupyter notebooken inneheld python-kode for generering av syntetiske data for ein populasjon som blei brukt som ein del av forarbeidet til prosjektet til DigDir Camparane i 2020.

## Naudsynte imports

In [ ]:
import pandas as pd
import numpy as np
import datetime
import random
import datetime

## Globale variablar

$$N = \text{Utvalg menn}$$
$$M = \text{Utvalg kvinner}$$

In [ ]:
N = 20
M = 20

## Globale Funksjonar

In [ ]:
# create function accepting a single parameter, the year as a four digit number
def get_random_date(startyear, endyear):
    """
    Select random date in range between two years.

    Input:
        startyear (int) - year in beginning of range
        endyear (int) - year at end of range

    Output:
        date (datetime) - random date in range.
    """
    year = random.randint(startyear, endyear)
    # try to get a date
    try:
        return datetime.datetime.strptime('{} {}'.format(random.randint(1, 366), year), '%j %Y')

    # if the value happens to be in the leap year range, try again
    except ValueError:
        return get_random_date(year)


def personnr():
    """
    Generate random personal number in the Norwegian style.

    Dependent on get_random_date function

    Output:
        int - personal number
    """
    fdato = get_random_date(1919, 2019)
    return str(fdato.strftime('%d%m%y')) + str(random.randint(10000, 99999))

def tlfnr():
    """Return phone no in the norwegian style."""
    return random.randint(10000000, 99999999)

def adr(post):
    """Generate random adress in the norwegian style."""
    tmp = post.sample(1, replace = True, axis=0)
    adresss = tmp["Postnummer"]
    city = tmp["Poststed"]
    return [adress, city]

## Globale klasser

In [ ]:
class vigo:
    """
    Class for vigo-data in data generation code.

    Methods:
        __init__() : initialize class with random variables
    """

    def __init__(self):
        """Initialize object with random data."""
        self.grades = {'RLE' : random.randint(1, 6),
                       'Naturfag' : random.randint(1, 6),
                       'Matematikk' : random.randint(1, 6)}
        self.absence = {'RLE' : abs(np.random.normal(0.05, 0.03)),
                        'Naturfag' : abs(np.random.normal(0.05, 0.03)),
                        'Matematikk' : abs(np.random.normal(0.05, 0.03))}

## Generer syntetiske navn

### Kjelder

Norske navn: https://no.wikipedia.org/wiki/Liste_over_norske_mannsnavn , https://no.wikipedia.org/wiki/Liste_over_norske_kvinnenavn og https://no.wikipedia.org/wiki/Liste_over_norske_etternavn

Postnummer: https://data.norge.no/datasets/5e6847ba-156d-4e14-85d3-8d7f8b727523

In [ ]:
data = pd.read_csv('navn.csv')
post = pd.read_csv('postnr.csv', converters={'Postnummer': lambda x: str(x)})

## Lag eit tilfeldig utvalg av menn

$$N = \text{Utvalg menn}$$

In [ ]:
mennFornavn = np.array(data["Malename"].sample(N, replace = True))
mennEtternavn = np.array(data["Lastname"].sample(N, replace = True))

## Lag eit tilfeldig utvalg av kvinner

$$M = \text{Utvalg kvinner}$$

In [ ]:
kvinnerFornavn = np.array(data["Femalename"].sample(M, replace = True))
kvinnerEtternavn = np.array(data["Lastname"].sample(M, replace = True))

## Kombiner til ein heil populasjon og lagre som JSON

In [ ]:
menn = pd.DataFrame({'Name' : mennFornavn, 'Lastname' : mennEtternavn})
kvinner = pd.DataFrame({'Name' : kvinnerFornavn, 'Lastname' : kvinnerEtternavn})

pnum = []
fnum = []
vig = []
adress = []
for _ in range(M+N):
    pnum.append(personnr())
    fnum.append(tlfnr())
    vig.append(vigo())
    adress.append("Test adresse " + str(random.randint(1, 100)))

if pd.Series(pnum).is_unique & pd.Series(fnum).is_unique:
    populasjon = pd.concat([menn, kvinner])
    populasjon['Personalno'] = pnum
    populasjon['Phoneno'] = fnum
    populasjon['Adress'] = adress
    populasjon[post.columns] = pd.concat([post.sample(M+N, replace = True, axis = 0)], axis=0, ignore_index=True)
    populasjon['VIGO'] = [{'Grade' : v.grades, 'Absence' : v.absence} for v in vig]
    populasjon = populasjon.reset_index(drop=True)
    populasjon = populasjon.set_index("Personalno")
    populasjon.to_json(r'populasjon.json')

else:
    print("Two or more identical personal numbers")